## **수상작 리뷰**

## 데이터셋 정보

**주제** : 교통사고 위험도 예측(regression)

**변수**
* curvature: 도로의 곡률 (연속형)

* lighting: 조명 상태 (범주형: 0=주간, 1=야간 등)

* weather: 날씨 상태 (범주형: clear, fog, rain 등)

* speed_limit: 제한속도 구간 (범주형 혹은 이산형)

* num_reported_accidents: 보고된 사고 건수 (이산형)


## 전처리
1. Mixed → Numerical 변환
* 범주형 변수를 원-핫 인코딩 또는 임베딩 형태로 변환.
* 완전한 수치화(모든 변수를 numerical로 변환)는 Kaggle 기준으로는 실패. 즉, RMSE는 낮지만 LB에서는 일반화가 안 됨.

2. Mixed → Categorical 변환
* 모든 피처를 범주형으로 처리해 CatBoost, TabM, FM(Factorization Machine) 등에서 사용.
* 이 방식이 신경망 기반 모델과 가장 잘 맞았음.

3. Latent Features (잠재 특성) 추가
* Autoencoder로 데이터를 잠재공간(latent space)에 매핑해 새로운 feature 생성.
* 또 다른 시도로 Genetic Programming(GP) 기반 수학적 조합 피처를 생성.
4. Feature Selection
* Lasso 회귀의 계수를 활용해 중요 피처 추출.
* 절댓값 0.05 이상인 계수만 남김으로써, 모델 해석력 향상 + 노이즈 제거.

## 인코딩 기법

* 범주형	Label Encoding / Embedding	(TabM, FM, CatBoost)
* 수치형	StandardScaler / 그대로 사용	(NN, Lasso)
* 혼합형	Categorical Encoding + Embedding	(Ensemble 단계)
* 잠재 특성	Autoencoder Representation	(추가 feature)
* 조합 특성	Genetic Programming	(Ensemble 시 추가)

## 모델링
1. Lasso Regression
* 빠르고 해석 가능한 baseline.
* CV와 LB 간의 차이가 거의 없음 → 모델 신뢰도 높음.
* 피처 중요도를 통해 문제 구조를 파악하는 데 핵심 역할.

2. Neural Networks (Keras, TabM, FM)
* PyTorch Tabular, Keras Embedding, FM(Factorization Machine) 등 다양한 구조 시도.
* 하지만 Kaggle LB 기준에서는 큰 성능 향상 없음.
* TabM (Masaya Kawamata의 노트북) 도입으로 표형 데이터용 NN 구조의 가능성을 봄.

3. Gradient Boosting (XGBoost, CatBoost)
* 초기에 LightGBM 시도 → 성능 미미.
* XGB + CatBoost 조합이 유의미한 개선을 줌.
* 특히 CatBoost의 baseline boosting 기능을 이용해 신경망 예측을 seed로 활용한 것이 주효.
* 즉, CatBoost가 “Residual Learner” 역할 수행.

4. Ensemble (Stacking + Hill Climbing)
* 다양한 모델을 혼합 : Lasso, TabM, Keras NN, CatBoost
* 최종적으로 이중 앙상블 구조:
* 1단계: 여러 개의 Keras + CatBoost 모델
* 2단계: Hill Climbing으로 최적 조합 탐색
* 점수 향상폭은 작지만 (0.00001~0.00002), 미세한 차이가 승부를 가름.

## 차별점 및 배울점
교재에서 활용하지 않은 Lasso Regression과 Neural Networks를 구현했다. 이를 통해 라쏘 회귀는 단순하지만 해석력과 피처 인사이트에서는 큰 장점을 가짐을 알 수 있었다.